#  Chatbot tes





In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import pandas as pd
import numpy as np
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [2]:
# Download NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nurul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nurul\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nurul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
with open('datset.json') as f:
  data1 = json.load(f)
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})
print(data)

                                                inputs  \
0                                                 Halo   
1                                                  Hai   
2                                                Halo!   
3                                         Selamat pagi   
4                                        Selamat siang   
..                                                 ...   
514  Apa tanda-tanda bahwa saya perlu mencari bantu...   
515  Bagaimana kesehatan mental saya dapat memengar...   
516  Apa yang bisa saya lakukan untuk mengurangi st...   
517  Mengapa penting untuk memantau perubahan kulit...   
518  Apa yang perlu saya perhatikan dalam pemantaua...   

                                        tags  
0                                      Salam  
1                                      Salam  
2                                      Salam  
3                                      Salam  
4                                      Salam  
..                   

In [11]:
#removing punctuations
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [4]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)


NameError: name 'x_train' is not defined

In [13]:
#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)

#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
17/17 [==============================] - 4s 18ms/step - loss: 5.4188 - accuracy: 0.0039
Epoch 2/200
17/17 [==============================] - 0s 11ms/step - loss: 5.4128 - accuracy: 0.0347
Epoch 3/200
17/17 [==============================] - 0s 11ms/step - loss: 5.4076 - accuracy: 0.0308
Epoch 4/200
17/17 [==============================] - 0s 12ms/step - loss: 5.3994 - accuracy: 0.0308
Epoch 5/200
17/17 [==============================] - 0s 13ms/step - loss: 5.3871 - accuracy: 0.0308
Epoch 6/200
17/17 [==============================] - 0s 13ms/step - loss: 5.3581 - accuracy: 0.0308
Epoch 7/200
17/17 [==============================] - 0s 12ms/step - loss: 5.3177 - accuracy: 0.0308
Epoch 8/200
17/17 [==============================] - 0s 11ms/step - loss: 5.2676 - accuracy: 0.0308
Epoch 9/200
17/17 [==============================] - 0s 11ms/step - loss: 5.1878 - accuracy: 0.0308
Epoch 10/200
17/17 [==============================] - 0s 11ms/step - loss: 5.0943 - accuracy: 0.0308

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

# #apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])




# Set parameters
vocab_size = 5000
embedding_dim = 64
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = len(data['inputs'])

# Create tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(data['inputs'])
word_index = tokenizer.word_index

# Create sequences
sequences = tokenizer.texts_to_sequences(data['inputs'])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Create training data
training_data = padded_sequences[:training_size]
training_labels = padded_sequences[:training_size]

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
num_epochs = 200
history = model.fit(training_data, y_train, epochs=num_epochs, verbose=2)

Epoch 1/200
17/17 - 4s - loss: 8.4654 - accuracy: 0.0135 - 4s/epoch - 260ms/step
Epoch 2/200
17/17 - 1s - loss: 7.1854 - accuracy: 0.0308 - 880ms/epoch - 52ms/step
Epoch 3/200
17/17 - 1s - loss: 5.6560 - accuracy: 0.0308 - 864ms/epoch - 51ms/step
Epoch 4/200
17/17 - 1s - loss: 5.4734 - accuracy: 0.0116 - 776ms/epoch - 46ms/step
Epoch 5/200
17/17 - 1s - loss: 5.4436 - accuracy: 0.0308 - 795ms/epoch - 47ms/step
Epoch 6/200
17/17 - 1s - loss: 5.4411 - accuracy: 0.0308 - 824ms/epoch - 48ms/step
Epoch 7/200
17/17 - 1s - loss: 5.4279 - accuracy: 0.0308 - 961ms/epoch - 57ms/step
Epoch 8/200
17/17 - 1s - loss: 5.4169 - accuracy: 0.0308 - 777ms/epoch - 46ms/step
Epoch 9/200
17/17 - 1s - loss: 5.4186 - accuracy: 0.0308 - 977ms/epoch - 57ms/step
Epoch 10/200
17/17 - 1s - loss: 5.4165 - accuracy: 0.0308 - 790ms/epoch - 46ms/step
Epoch 11/200
17/17 - 1s - loss: 5.4154 - accuracy: 0.0308 - 616ms/epoch - 36ms/step
Epoch 12/200
17/17 - 1s - loss: 5.4105 - accuracy: 0.0308 - 928ms/epoch - 55ms/step
Epo

In [11]:
import random
while True:
  texts_p = []
  prediction_input = input('You : ')
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)
  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()
  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print("Going Merry : ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

NameError: name 'input_shape' is not defined